In [1]:
import numpy as np
import pandas as pd
import time
import networkx as nx
import matplotlib.pyplot as plt

In [2]:
### next check the SiouxFalls
instance_name = "SiouxFalls"

In [3]:
node_attr = f'network_data/{instance_name}/SiouxFalls_node.tntp'

node_attr = pd.read_csv(node_attr, skiprows=1, sep='\t', header=None, names=["Node","X","Y",";"])

node_attr = node_attr.drop(columns=[";"])

node_attr.head()

node_attr.to_csv(f'network_data/{instance_name}/network_node.csv', index=False)

In [4]:
filepath = f'network_data/{instance_name}/SiouxFalls_net.tntp'

In [5]:
net = pd.read_csv(filepath, skiprows=8, sep='\t')
# drop columns named "~", ";"
net = net.drop(columns=["~", ";"])

net

,init_node,term_node,capacity,length,free_flow_time,b,power,speed,toll,link_type
0,1,2,25900.200640,6,6,0.15,4,0,0,1
1,1,3,23403.473190,4,4,0.15,4,0,0,1
2,2,1,25900.200640,6,6,0.15,4,0,0,1
3,2,6,4958.180928,5,5,0.15,4,0,0,1
4,3,1,23403.473190,4,4,0.15,4,0,0,1
...,...,...,...,...,...,...,...,...,...,...
71,23,22,5000.000000,4,4,0.15,4,0,0,1
72,23,24,5078.508436,2,2,0.15,4,0,0,1
73,24,13,5091.256152,4,4,0.15,4,0,0,1
74,24,21,4885.357564,3,3,0.15,4,0,0,1


In [6]:
net["Edge"] = net.index
net1 = net[["Edge","init_node","term_node","free_flow_time","capacity","b","power"]]

In [7]:
# column names are: Edge,Node_F,Node_T,FFTT,Capacity,Beta,Alpha
net1.columns = ["Edge","Node_F","Node_T","FFTT","Capacity","Beta","Alpha"]

In [8]:
net1.to_csv(f'network_data/{instance_name}/network_net.csv', index=False)

In [9]:
od = pd.read_csv(f'network_data/{instance_name}/SiouxFalls_od.csv')
od.to_csv(f'network_data/{instance_name}/network_od0.csv', index=False)
od

,O,D,Ton
0,2,1,100.0
1,3,1,100.0
2,4,1,500.0
3,5,1,200.0
4,6,1,300.0
...,...,...,...
523,19,24,100.0
524,20,24,400.0
525,21,24,500.0
526,22,24,1100.0


In [11]:
so_sol = pd.read_csv(f'network_data/{instance_name}/SiouxFalls_flow.tntp', skiprows=1, sep='\t', header=None, names=["init_node","term_node","flow","cost"])

so_sol

,init_node,term_node,flow,cost
0,1,2,4494.657646,6.000816
1,1,3,8119.079948,4.008691
2,2,1,4519.079948,6.000834
3,2,6,5967.336396,6.573598
4,3,1,8094.657646,4.008587
...,...,...,...,...
71,23,22,9626.210200,12.243138
72,23,24,7902.983927,3.759304
73,24,13,11112.394731,17.617021
74,24,21,10259.524716,11.752579


In [13]:
fftt = net["free_flow_time"].to_numpy()
capacity = net["capacity"].to_numpy()
beta = net["b"].to_numpy()
alpha = net["power"].to_numpy()

In [19]:
# fftt*self.link_flow + capacity/(self.alpha+1)*self.fftt*self.beta*(self.link_flow/self.capacity)**(self.alpha+1)

so_sol1 = fftt*so_sol["flow"] + capacity/(alpha+1)*fftt*beta*(so_sol["flow"]/capacity)**(alpha+1)

In [20]:
so_sol1.sum()

4231335.28710744